In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io

MAIN MISSION: See if it is possible to accuratly predict in which genre is a painting in, if we compare the pictures in certain ways (E.g. colors, contrasts, light)

TODO:
Kuidas ekstraheerida piltidest suvalised ruudud? \
Algoritm: \
    Loeme pildi sisse pikslitega\
    Loome ruudu (kui palju?)\
    Valime ruutude kaupa piltidelt pikslite väärtusi\
    Loeme sisse valitud ruudud (mis on nende kontrastid ruudu siseselt ja ruutude vahel, heledus ja tumedus, valguse kasutamine jne)\

Kuidas kasutame Skimage'it?
    

TODO: Kuidas lugeda sisse erinevat informatsiooni?\
Millist informatsiooni saame erinevatest värviväärtusest? \
Millist informatsiooni annab meile võrdlused ruutude sees ja ruutude vahel?  (Lähikontrast ja üleüldine kontrast?) 

SKIMAGE!\
\
3D numpy array:
  * y coordinate
  * x coordinate
  * RGB (in this sequence)

In [17]:
artists=pd.read_csv("artists.csv",index_col=["id"])

In [42]:
image='images/images/Caravaggio/Caravaggio_1.jpg'
resize ='resized/resized/Caravaggio_1.jpg'
img = skimage.io.imread(fname=image)
res = skimage.io.imread(fname=resize)
print(str(len(img)) + " " +str(len(img[0])) + " " +str(len(img[0][0])) + " ")
print(str(len(res)) + " " +str(len(res[0])) + " " +str(len(res[0][0])) + " ")
image='images/images/Caravaggio/Caravaggio_2.jpg'
resize ='resized/resized/Caravaggio_2.jpg'
img = skimage.io.imread(fname=image)
res = skimage.io.imread(fname=resize)
print(str(len(img)) + " " +str(len(img[0])) + " " +str(len(img[0][0])) + " ")
print(str(len(res)) + " " +str(len(res[0])) + " " +str(len(res[0][0])) + " ")

1000 691 3 
600 415 3 
1190 871 3 
714 523 3 


In [74]:
#Jagame pildi gridiks sellepärast, et saaksime adekvaatselt valida küll suvaliselt ruute sellest gridist, aga et need ei korduks. Vot
#Märgime ära gridi kordinaatide ühised nimetajad, mille järgi saab pildil navigeerida (nt (0,0), mis on esimene ruut vasakul üleval nurgas)
#Kasutame piltide resized versioone, hoida kokku andmete pealt
import random
used_squares=[]
while len(used_squares)<25:
    x = len(res)/20
    y = len(res[0])/20
    rand_y=random.randint(0,19)
    rand_x=random.randint(0,19)
    if (rand_y,rand_x) not in used_squares:
        used_squares.append((rand_y,rand_x))
used_squares
#Now we have the random starting points to the squares which we can use

[(9, 16),
 (2, 9),
 (16, 11),
 (8, 3),
 (0, 13),
 (9, 6),
 (14, 3),
 (6, 8),
 (10, 11),
 (8, 9),
 (15, 11),
 (0, 4),
 (17, 19),
 (4, 7),
 (15, 19),
 (0, 11),
 (14, 5),
 (13, 12),
 (15, 1),
 (19, 5),
 (7, 19),
 (19, 2),
 (14, 11),
 (9, 8),
 (18, 8)]

In [75]:
#squares_with_info is a 2D array, because it's not important for us to know where the square was, but which pixels were in one square
squares_with_info=[]
for start in used_squares:
    for i in range(20):
        one_row_of_pixels=[]
        for j in range(20):
            one_row_of_pixels.append(((res[start[0]*20+i][start[1]*20+j][0]),(res[start[0]*20+i][start[1]*20+j][1]),(res[start[0]*20+i][start[1]*20+j][2])))
        squares_with_info.append(one_row_of_pixels)
squares_with_info

[[(206, 151, 112),
  (184, 130, 96),
  (147, 96, 69),
  (122, 75, 57),
  (116, 73, 64),
  (106, 68, 65),
  (89, 58, 56),
  (80, 51, 53),
  (74, 48, 47),
  (75, 51, 47),
  (79, 54, 49),
  (82, 57, 50),
  (87, 58, 52),
  (92, 61, 56),
  (101, 68, 63),
  (106, 73, 66),
  (106, 72, 62),
  (98, 66, 55),
  (92, 59, 50),
  (86, 56, 48)],
 [(186, 132, 98),
  (161, 110, 83),
  (132, 84, 62),
  (117, 71, 56),
  (109, 68, 62),
  (103, 65, 64),
  (87, 55, 56),
  (73, 43, 45),
  (75, 49, 48),
  (74, 49, 45),
  (76, 51, 46),
  (82, 54, 50),
  (87, 58, 52),
  (92, 63, 57),
  (100, 69, 64),
  (109, 74, 68),
  (114, 77, 68),
  (111, 73, 62),
  (108, 71, 62),
  (101, 67, 58)],
 [(170, 119, 90),
  (139, 91, 69),
  (120, 74, 58),
  (112, 68, 59),
  (104, 65, 60),
  (103, 67, 67),
  (94, 59, 63),
  (70, 40, 42),
  (74, 46, 45),
  (70, 42, 39),
  (70, 42, 38),
  (76, 49, 42),
  (84, 55, 51),
  (91, 62, 58),
  (100, 69, 66),
  (110, 75, 71),
  (120, 80, 70),
  (122, 80, 68),
  (122, 82, 72),
  (113, 75, 66)]